In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
# Initialize the WebDriver with incognito mode (optional)

def get_main_html(link=None):
# Open the target website
    if link:
        options = Options()
        options.add_argument("--headless")
        driver = webdriver.Chrome(options=options)
        driver.get(link)
        # Click the "load more" button until it's no longer available
        while True:
            try:
                # Wait for the button to be clickable and then click it
                button = WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[@class='inline-flex justify-center items-center umd:py-2 cursor-pointer']"))
                )
                button.click()
                # Add a short delay after clicking the button
                time.sleep(1)
            except:
                # Break the loop if the button is not found or not clickable
                break

        time.sleep(5)

        # Optionally, you can wait for a specific element to ensure the page has finished loading
        
        # Find the target element and get its inner HTML
        page_html = driver.page_source


        # Close the browser when done
        driver.quit()
        return page_html


In [2]:
article_links = []

In [3]:
p = []

In [4]:
my_links = ['https://nhathuoclongchau.com.vn/bai-viet/me-va-be/ke-hoach-mang-thai','https://nhathuoclongchau.com.vn/bai-viet/me-va-be/mang-thai','https://nhathuoclongchau.com.vn/bai-viet/me-va-be/sinh-con','https://nhathuoclongchau.com.vn/bai-viet/me-va-be/cham-soc-be']

for link in my_links:
# Parse the HTML content with BeautifulSoup
    print(link)
    page_html = get_main_html(link)
    p.append(page_html)
    page_html = BeautifulSoup(page_html, 'html.parser')
    html_content = page_html.find('div', class_='grid grid-cols-1 omd:grid-cols-4 gap-4 omd:gap-5 umd:pt-4')
    domain = page_html.find('h1', class_='text-display3 umd:text-heading1 text-gray-7 font-bold')
    boxes = html_content.find_all('div', class_='omd:bg-white omd:p-4 omd:pb-5 overflow-hidden rounded-lg omd:rounded-xl')
    print(len(boxes))
    # Print the number of boxes found and their content
    links = []
    for box in boxes:
        links.append((box.find('a')['href'],domain))
    print(len(links))
    article_links.extend(links)

https://nhathuoclongchau.com.vn/bai-viet/me-va-be/ke-hoach-mang-thai
466
466
https://nhathuoclongchau.com.vn/bai-viet/me-va-be/mang-thai
2860
2860
https://nhathuoclongchau.com.vn/bai-viet/me-va-be/sinh-con
800
800
https://nhathuoclongchau.com.vn/bai-viet/me-va-be/cham-soc-be
4729
4729


In [5]:
root_link = 'https://nhathuoclongchau.com.vn'
for i in range(len(article_links)):
    article_links[i] = list(article_links[i])
    article_links[i][0] = root_link+article_links[i][0]

In [6]:
import json

In [7]:
# 1. Import thư viện cần thiết
from bs4 import BeautifulSoup
import json
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
retry_strategy = Retry(
    total=3,  # Số lần thử lại tối đa
    backoff_factor=1,  # Thời gian chờ giữa các lần thử lại (1 giây, 2 giây, 4 giây,...)
    status_forcelist=[429, 500, 502, 503, 504],  # Mã trạng thái yêu cầu retry
    allowed_methods=["HEAD", "GET", "OPTIONS"]  # Chỉ áp dụng retry với các phương thức này
)

adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)

In [8]:
def parse_html(link, domain, i=0):
    try:
        response = http.get(link, timeout=20)
        soup = BeautifulSoup(response.content, "html.parser")


        title = soup.find('h1',class_='text-text-primary font-semibold text-heading2 md:text-title2').text.strip().replace('\n', '').replace('\xa0', ' ').replace('\r', '').replace('  ', ' ')
        intro = soup.find('div', class_='mt-5 mb-6 md:mb-7').text.strip().replace('\n', '').replace('\xa0', ' ').replace('\r', '').replace('  ', ' ')

        # Lấy nội dung của từng phần
        html_content = soup.find('div', class_='mt-5 mb-6 md:mb-7').find_next_sibling()
        if not html_content:
            return None

        # Hàm trích xuất nội dung của từng phần
        def extract_section_content(tag):
            content = []
            for sibling in tag.find_next_siblings():
                if sibling.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                    break  # Kết thúc khi gặp tiêu đề mới
                if sibling.name == 'p':
                    content.append(sibling.text.strip())
                elif sibling.name in ['ul', 'ol']:
                    list_items = [li.text.strip() for li in sibling.find_all('li')]
                    list_type = 'list' if sibling.name == 'ul' else 'ordered_list'
                    content.append({list_type: list_items})
                elif sibling.name == 'blockquote':
                    content.append({'quote': sibling.text.strip()})
            return content

        # Trích xuất các mục chính và lồng nhau
        sections = []
        current_hierarchy = {2: None, 3: None, 4: None, 5: None, 6: None}

        for section in html_content.find_all(['h2', 'h3', 'h4', 'h5', 'h6']):
            level = int(section.name[1])  # Lấy số từ tên thẻ (h2 -> 2, h3 -> 3,...)
            section_data = {
                'heading': section.text.strip(),
                'content': extract_section_content(section)
            }

            if level == 2:
                sections.append(section_data)
                current_hierarchy[2] = sections[-1]
            else:
                parent_section = current_hierarchy[level - 1]
                if 'subsections' not in parent_section:
                    parent_section['subsections'] = []
                parent_section['subsections'].append(section_data)
            current_hierarchy[level] = section_data

        # Chuyển đổi thành JSON
        data = {
            "title": title,
            "introduction": intro,
            "sections": sections,
            "source_link": link,
            "domain": domain
        }

        # Lưu vào file JSON
        with open(f'data/article_data{i}.json', 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)

        return True
    except:
        return False

In [14]:
article_links[4601][1]

<h1 class="text-display3 umd:text-heading1 text-gray-7 font-bold">Chăm sóc bé</h1>

In [9]:
j = 1
miss =[]
for i in range(len(article_links)):
    print(j)
    j +=1
    try:
        tmp = parse_html(article_links[i][0],article_links[i][1],i)
    except:
        miss.append((article_links[i][0],article_links[i][1]))
        print(article_links[i][0],article_links[i][1])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [17]:
import os

# Đường dẫn đến thư mục chứa các file JSON
directory_path = 'data/'

# Lặp qua tất cả các file trong thư mục
for filename in os.listdir(directory_path):
    # Chỉ sửa các file có đuôi .json
    if filename.endswith('.json'):
        file_path = os.path.join(directory_path, filename)
        
        # Mở file để đọc và thêm nội dung vào cuối
        with open(file_path, 'r+', encoding='utf-8') as file:
            content = file.read().strip()  # Đọc nội dung và loại bỏ khoảng trắng
            
            # Thêm chuỗi '""}' vào cuối nội dung nếu chưa có
            if not content.endswith('""}'):
                file.seek(0)  # Quay lại đầu file
                file.write(content + '""}')  # Ghi nội dung cũ cùng với chuỗi bổ sung vào cuối
            print(f"Updated file: {file_path}")


Updated file: data/article_data0.json
Updated file: data/article_data1.json
Updated file: data/article_data10.json
Updated file: data/article_data100.json
Updated file: data/article_data1000.json
Updated file: data/article_data1001.json
Updated file: data/article_data1002.json
Updated file: data/article_data1003.json
Updated file: data/article_data1004.json
Updated file: data/article_data1005.json
Updated file: data/article_data1006.json
Updated file: data/article_data1007.json
Updated file: data/article_data1008.json
Updated file: data/article_data1009.json
Updated file: data/article_data101.json
Updated file: data/article_data1010.json
Updated file: data/article_data1011.json
Updated file: data/article_data1012.json
Updated file: data/article_data1013.json
Updated file: data/article_data1014.json
Updated file: data/article_data1015.json
Updated file: data/article_data1016.json
Updated file: data/article_data1017.json
Updated file: data/article_data1018.json
Updated file: data/article

In [18]:
# Danh sách domains và index_list
domains = ["Kế hoạch mang thai", "Mang thai", "Sinh con", "Chăm sóc bé"]
index_list = [465, 3325, 4125, 8854]  # Chỉ mục cuối cùng của từng domain

# Hàm xác định domain dựa trên index
def get_domain_for_index(index):
    for i, max_index in enumerate(index_list):
        if index <= max_index:
            return domains[i]
    return None

# Sửa các file JSON
for i in range(len(article_links)):  # Lặp qua các file đã tạo
    file_path = f'data/article_data{i}.json'
    try:
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

        # Cập nhật domain vào file JSON
        data['domain'] = get_domain_for_index(i)

        # Lưu lại file JSON
        with open(file_path, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)
            
        print(f"Updated domain for file: article_data{i}.json")
    except Exception as e:
        print(f"Error updating file {file_path}: {e}")

Updated domain for file: article_data0.json
Updated domain for file: article_data1.json
Updated domain for file: article_data2.json
Updated domain for file: article_data3.json
Updated domain for file: article_data4.json
Updated domain for file: article_data5.json
Updated domain for file: article_data6.json
Updated domain for file: article_data7.json
Updated domain for file: article_data8.json
Updated domain for file: article_data9.json
Updated domain for file: article_data10.json
Updated domain for file: article_data11.json
Updated domain for file: article_data12.json
Updated domain for file: article_data13.json
Updated domain for file: article_data14.json
Updated domain for file: article_data15.json
Updated domain for file: article_data16.json
Updated domain for file: article_data17.json
Updated domain for file: article_data18.json
Updated domain for file: article_data19.json
Updated domain for file: article_data20.json
Updated domain for file: article_data21.json
Updated domain for f

In [19]:
import os

# Đường dẫn đến thư mục chứa các file JSON
directory_path = 'data/'

# Đếm số lượng file .json trong thư mục
json_files = [filename for filename in os.listdir(directory_path) if filename.endswith('.json')]
json_file_count = len(json_files)

print(f"Tổng số file JSON trong thư mục '{directory_path}': {json_file_count}")


Tổng số file JSON trong thư mục 'data/': 8827
